# Beatmap Conversion Test

## Hit Object Syntax
x,y,time,type,hitSound,objectParams,hitSample

Hold syntax: x,y,time,type,hitSound,endTime:hitSample

### Parameters
* x (Integer) and y (Integer): Position in osu! pixels of the object.
    * 0: Hit circle
    * 1: Slider
    * 3: Spinner
    * 7: osu!mania hold
* time (Integer): Time when the object is to be hit, in milliseconds from the beginning of the beatmap's audio.
* type (Integer): Bit flags indicating the type of the object. See the type section.
* hitSound (Integer): Bit flags indicating the hitsound applied to the object. See the hitsounds section.
* objectParams (Comma-separated list): Extra parameters specific to the object's type.
* hitSample (Colon-separated list): Information about which samples are played when the object is hit. It is closely related to hitSound; see the hitsounds section. If it is not written, it defaults to 0:0:0:0:.

### Hold-note exclusive parameters
* endTime (Integer): End time of the hold, in milliseconds from the beginning of the beatmap's audio.
* x determines the index of the column that the hold will be in. It is computed by floor(x * columnCount / 512) and clamped between 0 and columnCount - 1.
* y does not affect holds. It defaults to the centre of the playfield, 192.

In [51]:
from pathlib import Path
import torch

In [2]:
fn = Path('data/er.osu')

In [52]:
def parse_beatmap(fn):
    with open(fn, mode='r', encoding='utf-8') as f:
        raw_content = f.read().splitlines()
    start_index = raw_content.index('[HitObjects]')
    beatmap = raw_content[start_index + 1:]

    obj_list = []
    xpos_list = set()
    xpos2num = {}

    for obj in beatmap:
        obj_split = obj.split(',')
        time = int(obj_split[2])
        xpos = int(obj_split[0])
        is_longnote = obj_split[3] != '1'
        end_time = obj_split[5].split(':', 1)[0] if is_longnote else 0

        obj_list.append([time, xpos, int(is_longnote), int(end_time)])
        xpos_list.add(xpos)

    xpos_list = sorted(xpos_list)
    xpos2num = {xpos: num for num, xpos in enumerate(xpos_list)}

    obj_list = [[obj[0], xpos2num[obj[1]], obj[2], obj[3]] for obj in obj_list]

    obj_tensor = torch.LongTensor(obj_list)

    return obj_tensor

In [56]:
beatmap = parse_beatmap(fn)
beatmap

tensor([[  840,     0,     0,     0],
        [  840,     1,     0,     0],
        [ 1150,     1,     0,     0],
        ...,
        [83391,     3,     0,     0],
        [83598,     0,     1, 85253],
        [83598,     2,     1, 85253]])

In [69]:
def convert_beatmap(beatmap:torch.Tensor, time_resolution:float):
    time_tensor = beatmap[:, 0]
    min_time_diff = torch.min(torch.diff(time_tensor.unique_consecutive()))
    scaling_factor = time_resolution / min_time_diff.item()
    quantized_tensor = torch.round(time_tensor * scaling_factor) * time_resolution

    return quantized_tensor

In [70]:
convert_beatmap(beatmap, time_resolution=10)

tensor([  820.,   820.,  1120.,  1420.,  2020.,  2420.,  2720.,  3020.,  3630.,
         4030.,  4330.,  4630.,  5640.,  5840.,  6040.,  6240.,  6440.,  6640.,
         6840.,  7040.,  7240.,  7240.,  7640.,  8050.,  8050.,  8450.,  8850.,
         8850.,  9250.,  9650.,  9650., 10060., 10460., 10460., 11060., 11260.,
        11260., 11860., 12060., 12060., 12470., 12870., 12870., 13070., 13270.,
        13470., 13670., 13670., 13870., 14070., 14270., 14470., 14670., 14880.,
        15080., 15280., 15280., 15480., 15680., 15880., 16080., 16280., 16480.,
        16680., 16880., 16880., 17090., 17390., 17690., 17690., 17990., 18290.,
        18490., 18490., 18890., 19300., 19300., 19500., 19700., 19900., 19900.,
        20500., 21710., 21710., 22510., 23110., 23310., 23310., 24120., 24920.,
        24920., 25720., 26330., 26530., 26530., 27330., 27330., 27730., 27730.,
        28130., 28330., 28530., 28740., 28940., 28940., 29340., 29540., 29740.,
        29940., 30140., 30340., 30540., 